# Redshift ML을 이용한 고객 구매 예측 모델

이 노트북에서는 Redshift ML을 사용하여 고객의 구매 금액을 예측하는 모델을 만들고 사용하는 방법을 보여줍니다.

In [ ]:
import boto3
import pandas as pd
from botocore.exceptions import ClientError

In [ ]:
# Redshift 데이터 API 클라이언트 생성
redshift_data = boto3.client('redshift-data')

# Redshift 연결 정보
cluster_identifier = "stoa-aiml-pipeline-workgroup"
database = "dev"
db_user = "admin"

In [ ]:
def execute_sql(sql):
    try:
        response = redshift_data.execute_statement(
            ClusterIdentifier=cluster_identifier,
            Database=database,
            DbUser=db_user,
            Sql=sql
        )
        query_id = response['Id']
        
        # 쿼리 완료 대기
        waiter = redshift_data.get_waiter('statement_finished')
        waiter.wait(Id=query_id)
        
        # 결과 가져오기
        result = redshift_data.get_statement_result(Id=query_id)
        
        return result
    except ClientError as e:
        print(f"An error occurred: {e}")

## 1. 데이터 확인

In [ ]:
sql = """
SELECT * FROM customers LIMIT 5;
"""

result = execute_sql(sql)
df = pd.DataFrame(result['Records'], columns=[col['name'] for col in result['ColumnMetadata']])
print(df)

## 2. Redshift ML 모델 생성

In [ ]:
sql = """
CREATE MODEL customer_purchase_model
FROM (
  SELECT age, city, season,
         purchase_amount as target
  FROM customers
)
TARGET target
FUNCTION predict_purchase_amount
IAM_ROLE default
AUTO OFF
MODEL_TYPE xgboost
PROBLEM_TYPE regression
OBJECTIVE 'reg:squarederror'
PREPROCESSORS 'one-hot-encoder'
HYPERPARAMETERS DEFAULT EXCEPT(
  max_depth '6',
  num_round '100'
)
"""

execute_sql(sql)
print("Model creation started. This may take a few minutes.")

## 3. 모델 상태 확인

In [ ]:
sql = """
SHOW MODEL customer_purchase_model;
"""

result = execute_sql(sql)
df = pd.DataFrame(result['Records'], columns=[col['name'] for col in result['ColumnMetadata']])
print(df)

## 4. 모델을 사용한 예측

In [ ]:
sql = """
SELECT age, city, season,
       purchase_amount as actual_amount,
       predict_purchase_amount(age, city, season) as predicted_amount
FROM customers
LIMIT 10;
"""

result = execute_sql(sql)
df = pd.DataFrame(result['Records'], columns=[col['name'] for col in result['ColumnMetadata']])
print(df)

## 5. 모델 성능 평가

In [ ]:
sql = """
SELECT AVG(POWER(actual - predicted, 2)) as mse,
       SQRT(AVG(POWER(actual - predicted, 2))) as rmse
FROM (
  SELECT purchase_amount as actual,
         predict_purchase_amount(age, city, season) as predicted
  FROM customers
);
"""

result = execute_sql(sql)
df = pd.DataFrame(result['Records'], columns=[col['name'] for col in result['ColumnMetadata']])
print(df)

## 6. 새로운 데이터에 대한 예측

In [ ]:
sql = """
SELECT predict_purchase_amount(35, 'Seoul', 'Q4') as predicted_amount;
"""

result = execute_sql(sql)
df = pd.DataFrame(result['Records'], columns=[col['name'] for col in result['ColumnMetadata']])
print(df)